In [1]:
import numpy as np
import h5py

### get PSSS center and FWHM

In [2]:
def get_PSS_data(bsread_file):
    channel_PSSS_center = "/data/SARFE10-PSSS059:FPICTURE.center/"
    channel_PSSS_fwhm = "/data/SARFE10-PSSS059:FPICTURE.fwhm/"
    with h5py.File(bsread_file, "r") as bsread:
        PSSS_center_data = bsread[f'{channel_PSSS_center}/data'][:]
        PSSS_center_pulseid = bsread[f'{channel_PSSS_center}/pulse_id'][:]
        PSSS_fwhm_data = bsread[f'{channel_PSSS_fwhm}/data'][:]
        PSSS_fwhm_pulseid = bsread[f'{channel_PSSS_fwhm}/pulse_id'][:]
    return PSSS_center_data, PSSS_center_pulseid, PSSS_fwhm_data, PSSS_fwhm_pulseid

### find pulse_id's of the hits from hitrate file

In [3]:
def get_online_hits(onlinehits_file):
    onlinehits_thresh = 10
    hits_online = []
    if onlinehits_file != None:
        OUTPUT_ONLINEHITS = True
        hits_file = open(onlinehits_file,'r')
        all_lines = hits_file.read().splitlines()
        for line in all_lines:
            (pulseid, frame, n_peaks_online, frame_good) = line.split()
            n_peaks_online = int(n_peaks_online)
            if n_peaks_online >= onlinehits_thresh:
                pulseid = int(pulseid)
                hits_online.append(pulseid)
        hits_file.close()
#    print(len(hits_online), " hits found online")
    return hits_online

### Order of frames(pulseid's) in hitrate file and raw file may be different, so order list  according to RAW file

In [4]:
def get_hits_online_sorted(data_file, hits_online=[]):
    hits_online_sorted = []
    with h5py.File(data_file, "r") as original_file:
        pulse_id_original = original_file["/data/JF06T32V01/pulse_id"][:]
        is_good_frame = original_file["/data/JF06T32V01/is_good_frame"][:]
        for i in range(len(pulse_id_original)):
            if not is_good_frame[i]:
                continue
            pulse_id = pulse_id_original[i][0]
            if pulse_id not in hits_online:
                continue
            hits_online_sorted.append(pulse_id)
    return hits_online_sorted        

### loop over converted file, find corresponding PSSS values via pulse_id, create new file

In [5]:
def produce_new_data_file(data_converted_file, hits_online_sorted, 
                         PSSS_center_data, PSSS_center_pulseid, PSSS_fwhm_data, PSSS_fwhm_pulseid):
    data_file_PSSS = data_converted_file[:-2] + "PSSS.h5"
    new_data_file = h5py.File(data_file_PSSS, "w")
    with h5py.File(data_converted_file, "r") as converted_file:
        images = converted_file["/data/data"]
        if len(images) != len(hits_online_sorted):
            print(f"Something very bad, stop. {len(images)} {len(hits_online_sorted)}")
        data_dset = new_data_file.create_dataset("/data/data", images.shape, dtype='float32')
        PSSS_center_dset = new_data_file.create_dataset("/PSSS/photon_energy_eV/mean", (len(images),), dtype='float32')
        PSSS_fwhm_dset = new_data_file.create_dataset("/PSSS/photon_energy_eV/width", (len(images),), dtype='float32')
        pulseid_dset = new_data_file.create_dataset("/data/pulse_id", (len(images),), dtype=np.uint64)
        for i in range(len(images)):    
            pulse_id = hits_online_sorted[i]
            image = images[i]
            if pulse_id in PSSS_center_pulseid:
                PSSS_center = PSSS_center_data[np.where(PSSS_center_pulseid == pulse_id)]
            else:
                print(f'{pulse_id} has no PSSS_center data, take some default value')
                print(PSSS_center_pulseid)
                PSSS_center = 5991.52
            if pulse_id in PSSS_fwhm_pulseid:
                PSSS_fwhm = PSSS_fwhm_data[np.where(PSSS_fwhm_pulseid == pulse_id)]
            else:
                print(f'{pulse_id} has no PSSS fwhm data, take some default value')
            data_dset[i] = image
            PSSS_center_dset[i] = PSSS_center
            PSSS_fwhm_dset[i] = PSSS_fwhm
            pulseid_dset[i] = pulse_id
    new_data_file.close()        

In [6]:
def add_PSSS_to_file(sample, run):
    bsread_file = f"/sf/alvra/data/p17805/raw/{sample}/run_000{run}.BSREAD.h5"
    onlinehits_file = f"/das/work/p17/p17805/res/jungfrau/hitrate/{sample}/run_000{run}.hitrate"
    data_file = f"/sf/alvra/data/p17805/raw/{sample}/run_000{run}.JF06T32V01.h5"
    data_converted_file = f"/das/work/p17/p17805/res/{sample}/data/dark/run_000{run}.JF06T32V01.h5"
    
    PSSS_center_data, PSSS_center_pulseid, PSSS_fwhm_data, PSSS_fwhm_pulseid = get_PSS_data(bsread_file)
    
    hits_online = get_online_hits(onlinehits_file)
    
    hits_online_sorted = get_hits_online_sorted(data_file, hits_online)
    
    produce_new_data_file(data_converted_file, hits_online_sorted, 
                         PSSS_center_data, PSSS_center_pulseid, PSSS_fwhm_data, PSSS_fwhm_pulseid)

In [7]:
sample="thaumatin8"
for run in range(113,134):
    print(run)
    add_PSSS_to_file(sample, run)

113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133


In [7]:
sample="thaumatin10"
for run in range(134,148):
    print(run)
    add_PSSS_to_file(sample, run)

134
135
136
137
138
139
140
141
142
143
144
145
146
147


In [7]:
sample="thaumatin10"
add_PSSS_to_file(sample, 140)